In [1]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.ndimage.measurements import label
from allFunction import *
from moviepy.editor import VideoFileClip
from IPython.display import HTML

C:\Users\ziaeeamir\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pickle_file=open('../code/svc.p','rb')#svc_pickle.p


dist_pickle = pickle.load(pickle_file)

# get attributes of our svc object
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
hog_channel=dist_pickle["hog_channel"]
color_space=dist_pickle["color_space"]
spatial_feat=dist_pickle['spatial_feat']
hog_feat=dist_pickle['hog_feat']
hist_feat=dist_pickle['hist_feat']
y_start_stop=dist_pickle['y_start_stop']
x_start_stop=dist_pickle['x_start_stop']
scale =  1.2
threshold=6



In [3]:
x_start_stop

[None, None]

In [4]:
y_start_stop=[350,695]

In [5]:
def pipline(image):
    windows = slide_window(image, x_start_stop=x_start_stop, y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))

    slide_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

    #window_img = draw_boxes(image, slide_windows, color=(0, 0, 255), thick=6)
    #plt.imshow(window_img)
    find_cars_windows=find_cars(image, y_start_stop[0], y_start_stop[1], scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    heat=np.zeros_like(image[:,:,0]).astype(np.float)
    heat=add_heat(heat,slide_windows+find_cars_windows)#find_cars_windows #,slide_windows
    heat=applay_threshold(heat,threshold)
    heatmap=np.clip(heat,0,255)
    labels=label(heatmap)
    draw_img=draw_labeled_bbox(image,labels)
    return draw_img


In [6]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
HTML("""
     <video width="960" height="600" controls>
     <source src="{0}" type="video/mp4">
     </video>
     """.format('../img/code/project_video.mp4'))

In [8]:
clip=VideoFileClip('../img/code/project_video.mp4')
clip=clip#.subclip(0,20)

In [9]:
new_clip= clip.fl_image(pipline)
new_clip.write_videofile('../img/code/result.mp4')

Moviepy - Building video ../img/code/result.mp4.
MoviePy - Writing audio in resultTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../img/code/result.mp4



Moviepy - Done !
Moviepy - video ready ../img/code/result.mp4


In [10]:
HTML("""
     <video width="960" height="600" controls>
     <source src="{0}" type="video/mp4">
     </video>
     """.format('../img/code/result.mp4'))